# Imports and Downloads


In [ ]:
!pip install remotezip tqdm opencv-python einops

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Created wheel for remotezip: filename=remotezip-0.12.1-py3-none-any.whl size=7933 sha256=bca4780313661a8ca733d6c5fab230c78003e0af444341b6e99b4a8a02f1759b
  Stored in directory: /root/.cache/pip/wheels/fc/76/04/beed1a6df4eb7430ee13c3900746edd517e5e597298d1f73f3
Successfully built remotezip


In [ ]:
# The way this tutorial uses the `TimeDistributed` layer requires TF>=2.10
!pip install -U "tensorflow>=2.10.0"
!pip install remotezip tqdm opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.

In [ ]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.9 MB/s eta 0:00:00
  Created wheel for pyyaml: filename=PyYAML-5.4.1-cp310-cp310-linux_x86_64.whl size=45658 sha256=71b1f8fda6517ca4f4e4ceb64e960d96ce05904d52ccddb74242e73654b2a407
  Stored in directory

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

RuntimeError: ignored

RuntimeError: ignored

# Video Download

In [ ]:
URL = 'https://storage.googleapis.com/29-06-fly-project/Action_v6.zip'

In [ ]:
def list_files_from_zip_url(zip_url):
  """ List the files in each class of the dataset given a URL with the zip file.

    Args:
      zip_url: A URL from which the files can be extracted from.

    Returns:
      List of files in each of the classes.
  """
  files = []
  with rz.RemoteZip(zip_url) as zip:
    for zip_info in zip.infolist():
      if zip_info.filename[0:3] != "__M":
        files.append(zip_info.filename)
  return files

In [ ]:
files = list_files_from_zip_url(URL)
files = [f for f in files if f.endswith('.avi')]
(files)

['Action_v6/v_Grooming_g01_c326.avi',
 'Action_v6/v_Pumping_g01_c119.avi',
 'Action_v6/v_Grooming_g01_c332.avi',
 'Action_v6/v_Pumping_g01_c125.avi',
 'Action_v6/v_Pumping_g01_c8.avi',
 'Action_v6/v_Pumping_g01_c131.avi',
 'Action_v6/v_Feeding_g01_c249.avi',
 'Action_v6/v_Feeding_g01_c261.avi',
 'Action_v6/v_Feeding_g01_c275.avi',
 'Action_v6/v_Grooming_g01_c124.avi',
 'Action_v6/v_Grooming_g01_c130.avi',
 'Action_v6/v_Pumping_g01_c327.avi',
 'Action_v6/v_Grooming_g01_c118.avi',
 'Action_v6/v_Grooming_g01_c36.avi',
 'Action_v6/v_Grooming_g01_c22.avi',
 'Action_v6/v_Pumping_g01_c333.avi',
 'Action_v6/v_Feeding_g01_c329.avi',
 'Action_v6/v_Feeding_g01_c315.avi',
 'Action_v6/v_Pumping_g01_c290.avi',
 'Action_v6/v_Feeding_g01_c96.avi',
 'Action_v6/v_Feeding_g01_c82.avi',
 'Action_v6/v_Feeding_g01_c301.avi',
 'Action_v6/v_Pumping_g01_c284.avi',
 'Action_v6/v_Feeding_g01_c69.avi',
 'Action_v6/v_Pumping_g01_c253.avi',
 'Action_v6/v_Feeding_g01_c55.avi',
 'Action_v6/v_Feeding_g01_c41.avi',
 'A

In [ ]:
def get_class(fname):
  """ Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Returns:
      Class that the file belongs to.
  """
  return fname.split('_')[-3]
get_class(files[10])

'Grooming'

In [ ]:
def get_files_per_class(files):
  """ Retrieve the files that belong to each class.

    Args:
      files: List of files in the dataset.

    Returns:
      Dictionary of class names (key) and files (values).
  """
  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class
get_files_per_class(files)

defaultdict(list,
            {'Grooming': ['Action_v6/v_Grooming_g01_c326.avi',
              'Action_v6/v_Grooming_g01_c332.avi',
              'Action_v6/v_Grooming_g01_c124.avi',
              'Action_v6/v_Grooming_g01_c130.avi',
              'Action_v6/v_Grooming_g01_c118.avi',
              'Action_v6/v_Grooming_g01_c36.avi',
              'Action_v6/v_Grooming_g01_c22.avi',
              'Action_v6/v_Grooming_g01_c291.avi',
              'Action_v6/v_Grooming_g01_c285.avi',
              'Action_v6/v_Grooming_g01_c252.avi',
              'Action_v6/v_Grooming_g01_c246.avi',
              'Action_v6/v_Grooming_g01_c247.avi',
              'Action_v6/v_Grooming_g01_c253.avi',
              'Action_v6/v_Grooming_g01_c284.avi',
              'Action_v6/v_Grooming_g01_c290.avi',
              'Action_v6/v_Grooming_g01_c23.avi',
              'Action_v6/v_Grooming_g01_c119.avi',
              'Action_v6/v_Grooming_g01_c37.avi',
              'Action_v6/v_Grooming_g01_c131.avi',
     

In [ ]:
NUM_CLASSES = 3
FILES_PER_CLASS = 412
files_for_class = get_files_per_class(files)
classes = list(files_for_class.keys())

In [ ]:
print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[2]]))

Num classes: 3
Num videos for class[0]: 412


In [ ]:
def select_subset_of_classes(files_for_class, classes, files_per_class):
  """ Create a dictionary with the class name and a subset of the files in that class.

    Args:
      files_for_class: Dictionary of class names (key) and files (values).
      classes: List of classes.
      files_per_class: Number of files per class of interest.

    Returns:
      Dictionary with class as key and list of specified number of video files in that class.
  """
  files_subset = dict()

  for class_name in classes:
    class_files = files_for_class[class_name]
    files_subset[class_name] = class_files[:files_per_class]

  return files_subset

In [ ]:
files_subset = select_subset_of_classes(files_for_class, classes[:NUM_CLASSES], FILES_PER_CLASS)
list(files_subset.keys())

['Grooming', 'Pumping', 'Feeding']

In [ ]:
def download_from_zip(zip_url, to_dir, file_names):
  """ Download the contents of the zip file from the zip URL.

    Args:
      zip_url: A URL with a zip file containing data.
      to_dir: A directory to download data to.
      file_names: Names of files to download.
  """
  with rz.RemoteZip(zip_url) as zip:
    for fn in tqdm.tqdm(file_names):
      class_name = get_class(fn)
      zip.extract(fn, str(to_dir / class_name))
      unzipped_file = to_dir / class_name / fn

      fn = pathlib.Path(fn).parts[-1]
      output_file = to_dir / class_name / fn
      unzipped_file.rename(output_file)

In [ ]:
def split_class_lists(files_for_class, count):
  """ Returns the list of files belonging to a subset of data as well as the remainder of
    files that need to be downloaded.

    Args:
      files_for_class: Files belonging to a particular class of data.
      count: Number of files to download.

    Returns:
      Files belonging to the subset of data and dictionary of the remainder of files that need to be downloaded.
  """
  split_files = []
  remainder = {}
  for cls in files_for_class:
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

In [ ]:
def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
  """ Download a subset of the UFC101 dataset and split them into various parts, such as
    training, validation, and test.

    Args:
      zip_url: A URL with a ZIP file with the data.
      num_classes: Number of labels.
      splits: Dictionary specifying the training, validation, test, etc. (key) division of data
              (value is number of files per split).
      download_dir: Directory to download data to.

    Return:
      Mapping of the directories containing the subsections of data.
  """
  files = list_files_from_zip_url(zip_url)
  files.remove(files[0])
  counter=0
  print(len(files))

  files_for_class = get_files_per_class(files)
  print(len(files_for_class['Pumping']))

  classes = list(files_for_class.keys())[:num_classes]
  for cls in classes:
    random.shuffle(files_for_class[cls])

  # Only use the number of classes you want in the dictionary
  files_for_class = {x: files_for_class[x] for x in classes}
  print(files_for_class)
  print(len(files_for_class['Pumping']))

  dirs = {}

  for split_name, split_count in splits.items():
     print(split_name, ":")
     split_dir = download_dir / split_name
     split_files, files_for_class = split_class_lists(files_for_class, split_count)

     print(split_name, len(split_files))
     download_from_zip(zip_url, split_dir, split_files)
     dirs[split_name] = split_dir

  return dirs

In [ ]:
download_dir = pathlib.Path('./Action_subset/')
subset_paths = download_ufc_101_subset(URL,
                                       num_classes = 3,
                                       splits = {"train": 247, "test": 165},
                                       download_dir = download_dir)

1236
412
{'Grooming': ['Action_v6/v_Grooming_g01_c44.avi', 'Action_v6/v_Grooming_g01_c85.avi', 'Action_v6/v_Grooming_g01_c95.avi', 'Action_v6/v_Grooming_g01_c30.avi', 'Action_v6/v_Grooming_g01_c27.avi', 'Action_v6/v_Grooming_g01_c187.avi', 'Action_v6/v_Grooming_g01_c36.avi', 'Action_v6/v_Grooming_g01_c174.avi', 'Action_v6/v_Grooming_g01_c195.avi', 'Action_v6/v_Grooming_g01_c111.avi', 'Action_v6/v_Grooming_g01_c150.avi', 'Action_v6/v_Grooming_g01_c327.avi', 'Action_v6/v_Grooming_g01_c249.avi', 'Action_v6/v_Grooming_g01_c259.avi', 'Action_v6/v_Grooming_g01_c178.avi', 'Action_v6/v_Grooming_g01_c216.avi', 'Action_v6/v_Grooming_g01_c125.avi', 'Action_v6/v_Grooming_g01_c286.avi', 'Action_v6/v_Grooming_g01_c10.avi', 'Action_v6/v_Grooming_g01_c84.avi', 'Action_v6/v_Grooming_g01_c374.avi', 'Action_v6/v_Grooming_g01_c151.avi', 'Action_v6/v_Grooming_g01_c391.avi', 'Action_v6/v_Grooming_g01_c17.avi', 'Action_v6/v_Grooming_g01_c163.avi', 'Action_v6/v_Grooming_g01_c181.avi', 'Action_v6/v_Grooming_g0

100%|██████████| 741/741 [03:52<00:00,  3.18it/s]


test :
test 495


100%|██████████| 495/495 [02:06<00:00,  3.92it/s]


# Frame Create

In [ ]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded.
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label.

      Args:
        path: Video file paths.
        n_frames: Number of frames.
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
fg = FrameGenerator(subset_paths['train'], 10, training=True)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 224, 224, 3)
Label: 1


# Transfer Learning

In [ ]:
batch_size = 8
num_frames = 8


gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

In [ ]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], num_frames, training = True),
                                          output_signature = output_signature)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature = output_signature)
test_ds = test_ds.batch(batch_size)

In [ ]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

True
True


In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

movinet_a0_base/
movinet_a0_base/checkpoint
movinet_a0_base/ckpt-1.data-00000-of-00001
movinet_a0_base/ckpt-1.index


In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 3)

In [ ]:
num_epochs = 2

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

Epoch 1/2
93/93 [==============================] - 9207s 99s/step - loss: 0.4437 - accuracy: 0.8394 - val_loss: 0.3516 - val_accuracy: 0.8606
Epoch 2/2
93/93 [==============================] - 9260s 100s/step - loss: 0.3033 - accuracy: 0.8853 - val_loss: 0.2880 - val_accuracy: 0.8909


# Model Evaluation


In [ ]:
model.evaluate(test_ds, return_dict=True)

62/62 [==============================] - 3780s 60s/step - loss: 0.3182 - accuracy: 0.8848


{'loss': 0.31819087266921997, 'accuracy': 0.8848484754562378}

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(subset_paths['train'], num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

62/62 [==============================] - 3651s 59s/step


# Save Model

In [ ]:
model.save('saved_model/vcv2')

In [ ]:
!zip -r /content/filev2.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/vcv2/ (stored 0%)
  adding: content/saved_model/vcv2/fingerprint.pb (stored 0%)
  adding: content/saved_model/vcv2/saved_model.pb (deflated 92%)
  adding: content/saved_model/vcv2/variables/ (stored 0%)
  adding: content/saved_model/vcv2/variables/variables.index (deflated 75%)
  adding: content/saved_model/vcv2/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: content/saved_model/vcv2/assets/ (stored 0%)
  adding: content/saved_model/vcv2/keras_metadata.pb (deflated 96%)


In [ ]:
while True:pass